<a href="https://colab.research.google.com/github/Dev2104/data-science/blob/main/weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1: Import libraires

In [ ]:
import requests #helps to fatch data from API
import pandas as pd #for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into trainng and testing sets
from sklearn.preprocessing import LabelEncoder #to convert catrgorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression tasks
from sklearn.metrics import mean_squared_error #to measure accurecy of prediction
from datetime import datetime, timedelta #to handle date and time
import pytz

In [ ]:
API_KEY = '7d13f891b95e1c47a78e6214d68b8e00' #my API key
BASE_URL = 'https://api.openweathermap.org/data/2.5/' #base URL


1. Fetch current weather Data

In [ ]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url) #get request to API
  data  = response.json()
  return{
      'city' : data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      'description' :data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'WindGustDir' : data['wind']['deg'],
      'WindGustSpeed' : data['wind']['speed'],
      'Pressure' : data['main']['pressure']


  }

2. read historical Data

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename) #load csv file into dataframe
  df = df.drop_duplicates() # remove rows with missing values
  return df

3. Prepare data for training

In [ ]:
def prepare_data(data):
  le = LabelEncoder() #crrate a labelencoder instance
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define the feature variables and target variables

  X = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #feature variables
  Y = data['RainTomorrow'] #target variable

  return X, Y, le #return all the variables that are assigned


4. Train rain prediction model

In [ ]:
def train_rain_model(X, Y):
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, Y_train) #train data

  Y_pred = model.predict(X_test) #to make prediction on test sets
  print("mean squered error for main model")
  print(mean_squared_error(Y_test, Y_pred))

  return model

5. Regression data

In [ ]:
def prepare_regression_data(data, feature):
 X, Y = [], [] #initiate list for feature and target values
 for i in range(len(data) - 1):
  X.append(data[feature].iloc[i])
  Y.append(data[feature].iloc[i+1])

  X= np.array(X).reshape(-1, 1)
  Y = np.array(Y)

  return X, Y

6. Train Regression Model

In [ ]:
def train_regression_model(X, Y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, Y)

  return model

7. Predict future

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array(predictions).reshape(-1, 1))
    predictions.append(next_value[0])

  return predictions[1:]

7. Weather analysis function

In [ ]:
from re import X
def weather_view():
  city = input("Enter city name: ")
  current_weather = get_current_weather(city)

  #load historical data

  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train rain prediction model

  X, Y, le = prepare_data(historical_data)
  rain_model = train_rain_model(X, Y)

  #map wind direction to compass points

  wind_deg = current_weather['WindGustDir'] % 360
  compass_points = [
    ("N", 348.75, 360), ("N", 0, 11.25),  # North appears in two ranges
    ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25), ("ENE", 56.25, 78.75),
    ("E", 78.75, 101.25), ("ESE", 101.25, 123.75), ("SE", 123.75, 146.25),
    ("SSE", 146.25, 168.75), ("S", 168.75, 191.25), ("SSW", 191.25, 213.75),
    ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75), ("W", 258.75, 281.25),
    ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25), ("NNW", 326.25, 348.75)
]


  compass_direction = next((point for point, start, end in compass_points if start <= wind_deg < end), "N")


  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1






  current_data = {
      'MinTemp' : current_weather['temp_min'],
      'MaxTemp' : current_weather['temp_max'],
      'WindGustDir' : compass_direction_encoded,
      'WindGustSpeed' : current_weather['WindGustDir'],
      'Humidity' : current_weather['humidity'],
      'Pressure' : current_weather['Pressure'],
      'Temp' : current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for temperatur and humidity

  X_temp, Y_temp = prepare_regression_data(historical_data, 'Temp')

  X_hum, Y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, Y_temp)

  hum_model = train_regression_model(X_hum, Y_hum)


  #predict future temp. & humidity

  future_temp = predict_future(temp_model, current_weather['current_temp'])

  future_humidity = predict_future(hum_model, current_weather['humidity'])


  #prepare time for future

  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  now_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  print(f"city: {city}, {current_weather['country']}")
  print(f"current temperature: {current_weather['current_temp']}°C") #remove celcius later if code doesnt work
  print(f"Feels like: {current_weather['feels_like']}°C")
  print(f"Minium Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"weather Prediction: {current_weather['description']}")
  print(f"Rain prediction: {'Yes' if rain_prediction else 'No'}")



  print("\nFuture Temperature Predictions")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")


weather_view()

Enter city name: England
mean squered error for main model
0.16216216216216217
city: England, US
current temperature: 19°C
Feels like: 20°C
Minium Temperature: 19°C
Maximum Temperature: 19°C
Humidity: 95%
weather Prediction: overcast clouds
Rain prediction: Yes

Future Temperature Predictions
15:00: 25.7°C
16:00: 25.7°C
17:00: 25.7°C
18:00: 25.7°C
19:00: 25.7°C

Future Humidity Predictions
15:00: 36.0%
16:00: 36.0%
17:00: 36.0%
18:00: 36.0%
19:00: 36.0%
